In [2]:
import time
from urllib.request import FancyURLopener  # This is library that helps us create the headless browser
from random import choice #This library helps pick a random item from a list
import bs4 as bs
import datetime
from datetime import timedelta, date
import pandas as pd
from selenium import webdriver
import sys
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
import numpy as np

C:\Users\cdubi\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
def init_browser():
    executable_path = {'executable_path': 'chromedriver.exe'}
    return Browser("chrome", **executable_path, headless=False)

In [4]:
user_agents = [
    'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',
    'Opera/9.80 (X11; Linux i686; Ubuntu/14.10) Presto/2.12.388 Version/12.16',
    'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201',
    'Mozilla/5.0 (Macintosh; Intel MaPc OS X 10_9_3) AppleWebKit/537.75.14 (KHTML, like Gecko) Version/7.0.3 Safari/7046A194A',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246'
]

class MyOpener(FancyURLopener, object):
    version = choice(user_agents)

# Get each article and their link
def getArticleMeta(infotable):
    for i in range(0,len(infotable)):
        CallDesc.append(infotable[i].getText().encode('utf-8'))
        CallLink.append(infotable[i]['href'])
        
def getDate(i):
    date = i.find("span", class_= "date")
    date = date.getText()
    try:
        date = pd.to_datetime(date,infer_datetime_format=True)
    except ValueError:
        if "Yesterday" in date:
            Date = datetime.date.today() - timedelta(1)
            date = date[10:]
            date = str(Date) + date
        else:
            date = date.split(',')[1]
            date = date + ' ' + str(2017)
        date = pd.to_datetime(date,infer_datetime_format=True)
    return date

#def getTicker(companyName):
#    df = pd.read_csv('company.txt', sep = '\t')
#    df = df[df['Country'] == 'USA']
#    dfD = df.set_index('Name')['Ticker'].to_dict()
#    del df
#    return dfD[process.extractOne(companyName, dfD.keys())[0]]

In [5]:
# This uses the search tool on Seeking Alpha to get pull specific company's transcript.
def getTranscriptBySearch(ticker):
    CallDesc = []
    CallLink = []
    browser = init_browser()
    try:
        ticker = ticker
        url = 'https://seekingalpha.com/search/transcripts?term='+ ticker
        browser.visit(url) # Start the browser and open 'url'
    except KeyError:
        print("That company doesn't exist in my masterlist")
        raise

    assert 'Seeking Alpha' in browser.title # Wait for the page to load
    html = driver.page_source # Get the html of the page
    driver.quit() # Close the browser

    soup = bs.BeautifulSoup(html, 'html.parser')

    infotable = soup.find_all("div", class_= "transcript_link")
    for i in infotable:
        date = getDate(i)
        if (date >= datetime.datetime(2016,9,1)) & (ticker in i.find('a').getText()):
            CallDesc.append(i.find('a').getText().encode('utf-8'))
            CallLink.append(i.find('a')['href'])
    CreateTextFiles(CallLink)

In [6]:
# get list of S&P 100 companies and tickers from wikipedia

import pandas as pd # library for data analysis
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML documents

# get the response in the form of html
wikiurl="https://en.wikipedia.org/wiki/S%26P_100"
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl)
print(response.status_code)

200


In [7]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(response.text, 'html.parser')
sandp100=soup.find_all('table')
df=pd.read_html(str(sandp100))

#convert list to dataframe
df=pd.DataFrame(df[2])
print(df.head(25))
df.to_csv('sandp100.csv', index=False)

   Symbol                          Name
0    AAPL                    Apple Inc.
1    ABBV                   AbbVie Inc.
2     ABT           Abbott Laboratories
3     ACN                     Accenture
4    ADBE                    Adobe Inc.
5     AIG  American International Group
6     ALL                      Allstate
7    AMGN                    Amgen Inc.
8     AMT                American Tower
9    AMZN                    Amazon.com
10    AXP              American Express
11     BA                    Boeing Co.
12    BAC          Bank of America Corp
13   BIIB                        Biogen
14     BK   The Bank of New York Mellon
15   BKNG              Booking Holdings
16    BLK                 BlackRock Inc
17    BMY          Bristol-Myers Squibb
18  BRK.B            Berkshire Hathaway
19      C                 Citigroup Inc
20    CAT              Caterpillar Inc.
21   CHTR        Charter Communications
22     CL             Colgate-Palmolive
23  CMCSA                 Comcast Corp.


In [8]:
# This goes into the transcript link and extracts the text from each link
def CreateTextFiles(CallLink):
    for link in range(0,len(CallLink)):
        myopener = MyOpener()
        page=myopener.open('https://seekingalpha.com' + CallLink[link])

        html = page.read()
        soup = bs.BeautifulSoup(html, 'lxml')

        infotable = soup.find_all("p", class_= "p")
        try:
            Output_File = open(str(infotable[0].find("a")['title']) + 
                               str(infotable[2].getText()) + ".txt",'w')
        except IndexError as exc:
            continue
        except TypeError as typ:
            continue

        for i in range(0,len(infotable)):
            if(infotable[i].getText().encode('utf-8').startswith('Copyright policy:')):
                break
            if(len(infotable[i].getText()) == 0):
                time.sleep(600)
                continue
            Output_File.write(infotable[i].getText().encode('utf-8'))
            Output_File.write("\n")
        Output_File.close()

In [108]:
url = 'https://seekingalpha.com/symbol/AAPL/transcripts'
browser = init_browser()
browser.visit(url)
time.sleep(10)
html = browser.html
soup = bs(html, "html.parser")
results = soup.find_all("article")
for result in results:
    title = result.find('h3').text
    url = result.find('a')['href']
    date = result.find('span').text
    date = pd.to_datetime(date,infer_datetime_format=True)
    if (date > datetime.datetime(2020, 1, 1)):
        print(title)
        print(url)
        print(date)
    
    
    
    


Apple Inc. (AAPL) CEO Tim Cook on Q4 2020 Results - Earnings Call Transcript
/article/4382943-apple-inc-aapl-ceo-tim-cook-on-q4-2020-results-earnings-call-transcript?source=content_type%3Areact%7Csection%3ATranscripts%7CsectionAsset%3ATranscripts%7Cfirst_level_url%3Asymbol%7Cbutton%3AAuthor%7Clock_status%3ANo%7Cline%3A1
2020-10-29 00:00:00
Apple Inc. (AAPL) CEO Tim Cook on Q3 2020 Results - Earnings Call Transcript
/article/4362707-apple-inc-aapl-ceo-tim-cook-on-q3-2020-results-earnings-call-transcript?source=content_type%3Areact%7Csection%3ATranscripts%7CsectionAsset%3ATranscripts%7Cfirst_level_url%3Asymbol%7Cbutton%3AAuthor%7Clock_status%3ANo%7Cline%3A2
2020-07-30 00:00:00
Apple Inc. (AAPL) CEO Tim Cook on Q2 2020 Results - Earnings Call Transcript
/article/4341792-apple-inc-aapl-ceo-tim-cook-on-q2-2020-results-earnings-call-transcript?source=content_type%3Areact%7Csection%3ATranscripts%7CsectionAsset%3ATranscripts%7Cfirst_level_url%3Asymbol%7Cbutton%3AAuthor%7Clock_status%3ANo%7Clin

In [9]:
def getTranscriptInfo(ticker, browser_):   
    url = 'https://seekingalpha.com/symbol/' + ticker + '/transcripts'
    browser = browser_
    browser.visit(url)
    time.sleep(30)
    html = browser.html
    soup = bs(html, "html.parser")
    results = soup.find_all("article")
    search_results = []
    for result in results:
        try:
            title = result.find('h3').text
            url = result.find('a')['href']
            date = result.find('span').text
            date = pd.to_datetime(date,infer_datetime_format=True)
            if (date > datetime.datetime(2020, 1, 1)):
                print(title)
                print(url)
                print(date)
                print('----------------------------------------')
                results_dict = {
                    "title": title,
                    "url": url,
                    "date": date,
                    "ticker": ticker
                }
                search_results.append(results_dict)
        except:
            continue
    return search_results

In [10]:
browser = init_browser()
getTranscriptInfo('AAPL', browser)

Apple Inc. (AAPL) CEO Tim Cook on Q4 2020 Results - Earnings Call Transcript
/article/4382943-apple-inc-aapl-ceo-tim-cook-on-q4-2020-results-earnings-call-transcript?source=content_type%3Areact%7Csection%3ATranscripts%7CsectionAsset%3ATranscripts%7Cfirst_level_url%3Asymbol%7Cbutton%3AAuthor%7Clock_status%3ANo%7Cline%3A1
2020-10-29 00:00:00
----------------------------------------
Apple Inc. (AAPL) CEO Tim Cook on Q3 2020 Results - Earnings Call Transcript
/article/4362707-apple-inc-aapl-ceo-tim-cook-on-q3-2020-results-earnings-call-transcript?source=content_type%3Areact%7Csection%3ATranscripts%7CsectionAsset%3ATranscripts%7Cfirst_level_url%3Asymbol%7Cbutton%3AAuthor%7Clock_status%3ANo%7Cline%3A2
2020-07-30 00:00:00
----------------------------------------
Apple Inc. (AAPL) CEO Tim Cook on Q2 2020 Results - Earnings Call Transcript
/article/4341792-apple-inc-aapl-ceo-tim-cook-on-q2-2020-results-earnings-call-transcript?source=content_type%3Areact%7Csection%3ATranscripts%7CsectionAsset%3

[{'title': 'Apple Inc. (AAPL) CEO Tim Cook on Q4 2020 Results - Earnings Call Transcript',
  'url': '/article/4382943-apple-inc-aapl-ceo-tim-cook-on-q4-2020-results-earnings-call-transcript?source=content_type%3Areact%7Csection%3ATranscripts%7CsectionAsset%3ATranscripts%7Cfirst_level_url%3Asymbol%7Cbutton%3AAuthor%7Clock_status%3ANo%7Cline%3A1',
  'date': Timestamp('2020-10-29 00:00:00')},
 {'title': 'Apple Inc. (AAPL) CEO Tim Cook on Q3 2020 Results - Earnings Call Transcript',
  'url': '/article/4362707-apple-inc-aapl-ceo-tim-cook-on-q3-2020-results-earnings-call-transcript?source=content_type%3Areact%7Csection%3ATranscripts%7CsectionAsset%3ATranscripts%7Cfirst_level_url%3Asymbol%7Cbutton%3AAuthor%7Clock_status%3ANo%7Cline%3A2',
  'date': Timestamp('2020-07-30 00:00:00')},
 {'title': 'Apple Inc. (AAPL) CEO Tim Cook on Q2 2020 Results - Earnings Call Transcript',
  'url': '/article/4341792-apple-inc-aapl-ceo-tim-cook-on-q2-2020-results-earnings-call-transcript?source=content_type%3Are

In [13]:
all_results = []
df = pd.read_csv('sandp100.csv', usecols = ['Symbol'])
tickers = df['Symbol'].tolist()
browser = init_browser()
for ticker in tickers:
    counter = 0
    results = getTranscriptInfo(ticker, browser)
    all_results = all_results + results
    time.sleep(20)
    
    

Apple Inc. (AAPL) CEO Tim Cook on Q4 2020 Results - Earnings Call Transcript
/article/4382943-apple-inc-aapl-ceo-tim-cook-on-q4-2020-results-earnings-call-transcript?source=content_type%3Areact%7Csection%3ATranscripts%7CsectionAsset%3ATranscripts%7Cfirst_level_url%3Asymbol%7Cbutton%3AAuthor%7Clock_status%3ANo%7Cline%3A1
2020-10-29 00:00:00
----------------------------------------
Apple Inc. (AAPL) CEO Tim Cook on Q3 2020 Results - Earnings Call Transcript
/article/4362707-apple-inc-aapl-ceo-tim-cook-on-q3-2020-results-earnings-call-transcript?source=content_type%3Areact%7Csection%3ATranscripts%7CsectionAsset%3ATranscripts%7Cfirst_level_url%3Asymbol%7Cbutton%3AAuthor%7Clock_status%3ANo%7Cline%3A2
2020-07-30 00:00:00
----------------------------------------
Apple Inc. (AAPL) CEO Tim Cook on Q2 2020 Results - Earnings Call Transcript
/article/4341792-apple-inc-aapl-ceo-tim-cook-on-q2-2020-results-earnings-call-transcript?source=content_type%3Areact%7Csection%3ATranscripts%7CsectionAsset%3

In [15]:
df_all = pd.DataFrame(all_results)

In [16]:
df_all.to_csv('transcript_list.csv', index=False)

In [31]:
def getTextFromArticle(url, browser_):
    url_begin = 'https://seekingalpha.com/'
    url_end = url
    browser = browser_
    browser.visit(url_begin+url_end)
    time.sleep(10)
    html = browser.html
    soup = bs(html, "html.parser")
    texts = soup.find_all('p')
    total_doc = ''
    for text in texts:
        total_doc += text.text + '\n'
    return total_doc


In [19]:
texts = soup.find_all('p')
total_doc = ''
for text in texts:
    total_doc += text.text + '\n'
print(total_doc)
    

Adobe Inc. (NASDAQ:ADBE) Q4 2020 Earnings Conference Call December 10, 2020 11:00 AM ET
Company Participants
Jonathan Vaas - Associate General Counsel and Vice President, Investor Relations
Shantanu Narayen - Chairman, President and Chief Executive Officer
Abhay Parasnis - Chief Technology Officer and Chief Product Officer, Document Cloud
Scott Belsky - Chief Product Officer and Executive Vice President, Creative Cloud
Anil Chakravarthy - Executive Vice President and General Manager, Digital Experience Business and Worldwide Field Operations
Gloria Chen - Chief People Officer and Executive Vice President, Employee Experience
John Murphy - Executive Vice President and Chief Financial Officer
Conference Call Participants
Brent Thill - Jefferies
Brad Zelnick - Credit Suisse
Mark Moerdler - Bernstein Research
Alex Zukin - RBC Capital Markets
Jennifer Lowe - UBS
Jay Vleeschhouwer - Griffin Securities
Kirk Materne - Evercore ISI
Keith Bachman - BMO Capital Markets
Ken Wong - Guggenheim Secur

In [30]:
df_all = pd.read_csv("transcript_list.csv")
df_ECT = df_all[df_all['title'].str.contains("Earnings Call Transcript")]
df_ECT["text"] = np.nan
df_ECT

C:\Users\cdubi\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,title,url,date,ticker,text
0,Apple Inc. (AAPL) CEO Tim Cook on Q4 2020 Resu...,/article/4382943-apple-inc-aapl-ceo-tim-cook-o...,2020-10-29,AAPL,NaN
1,Apple Inc. (AAPL) CEO Tim Cook on Q3 2020 Resu...,/article/4362707-apple-inc-aapl-ceo-tim-cook-o...,2020-07-30,AAPL,NaN
2,Apple Inc. (AAPL) CEO Tim Cook on Q2 2020 Resu...,/article/4341792-apple-inc-aapl-ceo-tim-cook-o...,2020-04-30,AAPL,NaN
3,Apple Inc. (AAPL) CEO Tim Cook on Q1 2020 Resu...,/article/4319666-apple-inc-aapl-ceo-tim-cook-o...,2020-01-28,AAPL,NaN
8,AbbVie Inc.'s (ABBV) CEO Rick Gonzalez on Q3 2...,/article/4383381-abbvie-inc-s-abbv-ceo-rick-go...,2020-10-30,ABBV,NaN
...,...,...,...,...,...
1031,"Walmart, Inc. (WMT) CEO Doug McMillon on Q1 20...",/article/4348814-walmart-inc-wmt-ceo-doug-mcmi...,2020-05-19,WMT,NaN
1035,Exxon Mobil Corporation (XOM) Q3 2020 Results ...,/article/4383411-exxon-mobil-corporation-xom-q...,2020-10-30,XOM,NaN
1037,Exxon Mobil Corp (XOM) Q2 2020 Results - Earni...,/article/4363013-exxon-mobil-corp-xom-q2-2020-...,2020-07-31,XOM,NaN
1039,Exxon Mobil Corporation (XOM) CEO Darren Woods...,/article/4342148-exxon-mobil-corporation-xom-c...,2020-05-01,XOM,NaN


In [32]:
df_ECT.iloc[0]['url']

'/article/4382943-apple-inc-aapl-ceo-tim-cook-on-q4-2020-results-earnings-call-transcript?source=content_type%3Areact%7Csection%3ATranscripts%7CsectionAsset%3ATranscripts%7Cfirst_level_url%3Asymbol%7Cbutton%3AAuthor%7Clock_status%3ANo%7Cline%3A1'

In [40]:
print('')

In [37]:
browser = init_browser()
text = getTextFromArticle(df_ECT.iloc[0]['url'],browser)




In [34]:
df_ECT.iloc[0]['text'] = getTextFromArticle(df_ECT.iloc[0]['url'],browser)

C:\Users\cdubi\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [36]:
df_ECT.head(20)

,title,url,date,ticker,text
0,Apple Inc. (AAPL) CEO Tim Cook on Q4 2020 Resu...,/article/4382943-apple-inc-aapl-ceo-tim-cook-o...,2020-10-29,AAPL,NaN
1,Apple Inc. (AAPL) CEO Tim Cook on Q3 2020 Resu...,/article/4362707-apple-inc-aapl-ceo-tim-cook-o...,2020-07-30,AAPL,NaN
2,Apple Inc. (AAPL) CEO Tim Cook on Q2 2020 Resu...,/article/4341792-apple-inc-aapl-ceo-tim-cook-o...,2020-04-30,AAPL,NaN
3,Apple Inc. (AAPL) CEO Tim Cook on Q1 2020 Resu...,/article/4319666-apple-inc-aapl-ceo-tim-cook-o...,2020-01-28,AAPL,NaN
8,AbbVie Inc.'s (ABBV) CEO Rick Gonzalez on Q3 2...,/article/4383381-abbvie-inc-s-abbv-ceo-rick-go...,2020-10-30,ABBV,NaN
10,AbbVie Inc. (ABBV) CEO Rick Gonzalez on Q2 202...,/article/4363134-abbvie-inc-abbv-ceo-rick-gonz...,2020-07-31,ABBV,NaN
11,AbbVie Inc. (ABBV) CEO Rick Gonzalez on Q1 202...,/article/4342124-abbvie-inc-abbv-ceo-rick-gonz...,2020-05-01,ABBV,NaN
12,AbbVie Inc. (ABBV) CEO Rick Gonzalez on Q4 201...,/article/4322499-abbvie-inc-abbv-ceo-rick-gonz...,2020-02-07,ABBV,NaN
14,Abbott Laboratories (ABT) CEO Robert Ford on Q...,/article/4380361-abbott-laboratories-abt-ceo-r...,2020-10-21,ABT,NaN
16,"Abbott Laboratories (ABT) CEO, Robert Ford on ...",/article/4358872-abbott-laboratories-abt-ceo-r...,2020-07-16,ABT,NaN


In [45]:
text = getTextFromArticle('/article/4338017-abbott-laboratories-abt-ceo-robert-ford-on-q1-2020-results-earnings-call-transcript', browser)

In [61]:
for index, row in df_ECT.iloc[150:].iterrows():
    try:
        text = getTextFromArticle(row['url'],browser)
        time.sleep(1)
        file = open(row['title'] + '.txt','w')
        time.sleep(1)
        file.writelines(text) 
        file.close()
    except:
        continue
    


In [60]:
df_ECT.iloc[150:]

,title,url,date,ticker,text
377,Ford Motor Company (F) CEO Jim Farley on Q3 20...,/article/4382475-ford-motor-company-f-ceo-jim-...,2020-10-29,F,NaN
385,Ford Motor Company's (F) CEO Jim Hackett on Q2...,/article/4362728-ford-motor-companys-f-ceo-jim...,2020-07-31,F,NaN
387,Facebook's (FB) CEO Mark Zuckerberg on Q3 2020...,/article/4382959-facebooks-fb-ceo-mark-zuckerb...,2020-10-30,FB,NaN
389,"Facebook, Inc. (FB) CEO Mark Zuckerberg on Q2 ...",/article/4362679-facebook-inc-fb-ceo-mark-zuck...,2020-07-30,FB,NaN
390,"Facebook, Inc. (FB) CEO Mark Zuckerberg on Q1 ...",/article/4341309-facebook-inc-fb-ceo-mark-zuck...,2020-04-29,FB,NaN
...,...,...,...,...,...
1031,"Walmart, Inc. (WMT) CEO Doug McMillon on Q1 20...",/article/4348814-walmart-inc-wmt-ceo-doug-mcmi...,2020-05-19,WMT,NaN
1035,Exxon Mobil Corporation (XOM) Q3 2020 Results ...,/article/4383411-exxon-mobil-corporation-xom-q...,2020-10-30,XOM,NaN
1037,Exxon Mobil Corp (XOM) Q2 2020 Results - Earni...,/article/4363013-exxon-mobil-corp-xom-q2-2020-...,2020-07-31,XOM,NaN
1039,Exxon Mobil Corporation (XOM) CEO Darren Woods...,/article/4342148-exxon-mobil-corporation-xom-c...,2020-05-01,XOM,NaN


In [ ]:
wikiurl="https://en.wikipedia.org/wiki/S%26P_100"
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl)
print(response.status_code)
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(response.text, 'html.parser')
sandp100=soup.find_all('table')
df=pd.read_html(str(sandp100))

#convert list to dataframe
df=pd.DataFrame(df[2])
print(df.head(25))
df.to_csv('sandp100.csv', index=False)

In [6]:
import pandas as pd # library for data analysis
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML documents

wikiurl2="https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
response=requests.get(wikiurl2)
print(response.status_code)
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(response.text, 'html.parser')
sandp500=soup.find_all('table')
df2=pd.read_html(str(sandp500))

#convert list to dataframe
#df2=pd.DataFrame(df[2])
#print(df.head(25))
#df.to_csv('sandp100.csv', index=False)

200


In [19]:
sandp100_df = pd.read_csv('sandp100.csv')
sandp500_df = df2[0]
sandp100_df_merged = sandp100_df.merge(sandp500_df, how='left', left_on='Symbol', right_on='Symbol').drop(columns=['Security', 'SEC filings'])
sandp100_df_merged.to_csv('sandp100_more_detail.csv', index=False)

In [ ]:
articles = df_ECT['url'].tolist()

tickers = df['Symbol'].tolist()